In [ ]:
import pandas as pd
import numpy as np
import os

data_dir = os.path.join(os.path.dirname(os.getcwd()), 'data')
if not os.path.isdir(data_dir):
    data_dir = os.path.join(os.getcwd(), 'data')

csv_files = sorted([
    f for f in os.listdir(data_dir)
    if f.endswith('.csv') and f.startswith('full_gnss') and f != 'full_gnss_e.csv'
])

stations = []
for f in csv_files:
    df = pd.read_csv(os.path.join(data_dir, f),
                     usecols=['X_Coord', 'Y_Coord', 'h_Coord'])
    name = f.replace('full_gnss_', '').replace('.csv', '')
    stations.append({
        'Tram': name,
        'X_mean (m)': df['X_Coord'].mean(),
        'Y_mean (m)': df['Y_Coord'].mean(),
        'h_mean (m)': df['h_Coord'].mean(),
        'So dong': len(df),
        'NaN (%)': df['h_Coord'].isna().mean() * 100,
    })

sdf = pd.DataFrame(stations)

# Phan loai nhom
def classify(name):
    if name.startswith('10'):
        return 'High West'
    elif name.endswith('e'):
        return 'East'
    else:
        return 'West'

sdf['Nhom'] = sdf['Tram'].apply(classify)
sdf = sdf[['Tram', 'Nhom', 'X_mean (m)', 'Y_mean (m)', 'h_mean (m)', 'So dong', 'NaN (%)']]

print(f'Tong so tram: {len(sdf)}')
print(f'  East:      {(sdf["Nhom"]=="East").sum()} tram')
print(f'  West:      {(sdf["Nhom"]=="West").sum()} tram')
print(f'  High West: {(sdf["Nhom"]=="High West").sum()} tram')
print()

pd.set_option('display.float_format', '{:.3f}'.format)
display(sdf)

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

east_mask = sdf['Nhom'] == 'East'
west_mask = sdf['Nhom'] == 'West'
high_mask = sdf['Nhom'] == 'High West'

# ====== Figure 1: 3D ======
fig = plt.figure(figsize=(16, 11))
ax = fig.add_subplot(111, projection='3d')

groups = [
    (east_mask, 'red',   'o', 'East (E)'),
    (west_mask, 'blue',  's', 'West (W)'),
    (high_mask, 'green', '^', 'High West (101w-105w)'),
]
for mask, color, marker, label in groups:
    sub = sdf[mask]
    ax.scatter(sub['X_mean (m)'], sub['Y_mean (m)'], sub['h_mean (m)'],
              c=color, marker=marker, s=200, alpha=0.9,
              edgecolors='black', linewidths=0.8, label=label, zorder=5)
    for _, row in sub.iterrows():
        ax.text(row['X_mean (m)'] + 15, row['Y_mean (m)'] + 15,
                row['h_mean (m)'] + 0.5, row['Tram'],
                fontsize=9, fontweight='bold', color=color)

# Noi cap E-W
for i in range(1, 7):
    e = sdf[sdf['Tram'] == f'{i}e']
    w = sdf[sdf['Tram'] == f'{i}w']
    if len(e) and len(w):
        e, w = e.iloc[0], w.iloc[0]
        ax.plot([e['X_mean (m)'], w['X_mean (m)']],
                [e['Y_mean (m)'], w['Y_mean (m)']],
                [e['h_mean (m)'], w['h_mean (m)']],
                'k--', alpha=0.3, linewidth=1)

# Noi tram cao voi tram chan
for i in range(1, 6):
    h_row = sdf[sdf['Tram'] == f'10{i}w']
    w_row = sdf[sdf['Tram'] == f'{i}w']
    if len(h_row) and len(w_row):
        h_r, w_r = h_row.iloc[0], w_row.iloc[0]
        ax.plot([h_r['X_mean (m)'], w_r['X_mean (m)']],
                [h_r['Y_mean (m)'], w_r['Y_mean (m)']],
                [h_r['h_mean (m)'], w_r['h_mean (m)']],
                'g:', alpha=0.5, linewidth=1.5)

ax.set_xlabel('X (m)', fontsize=12, labelpad=10)
ax.set_ylabel('Y (m)', fontsize=12, labelpad=10)
ax.set_zlabel('h – Do cao (m)', fontsize=12, labelpad=10)
ax.set_title('Vi tri 3D cac tram GNSS (17 tram)', fontsize=15, fontweight='bold')
ax.legend(fontsize=11, loc='upper left')
ax.view_init(elev=25, azim=-60)
plt.tight_layout()
plt.show()

# ====== Figure 2: Mat bang 2D ======
fig2, ax2 = plt.subplots(figsize=(14, 10))

for mask, color, marker, label in groups:
    sub = sdf[mask]
    ax2.scatter(sub['X_mean (m)'], sub['Y_mean (m)'],
               c=color, marker=marker, s=250, alpha=0.9,
               edgecolors='black', linewidths=1, label=label, zorder=5)
    for _, row in sub.iterrows():
        ax2.annotate(
            f"{row['Tram']}\n({row['h_mean (m)']:.1f}m)",
            (row['X_mean (m)'], row['Y_mean (m)']),
            textcoords='offset points', xytext=(12, 8),
            fontsize=9, fontweight='bold', color=color,
            bbox=dict(boxstyle='round,pad=0.2', facecolor='white', alpha=0.7))

for i in range(1, 7):
    e = sdf[sdf['Tram'] == f'{i}e']
    w = sdf[sdf['Tram'] == f'{i}w']
    if len(e) and len(w):
        e, w = e.iloc[0], w.iloc[0]
        ax2.plot([e['X_mean (m)'], w['X_mean (m)']],
                 [e['Y_mean (m)'], w['Y_mean (m)']],
                 'k--', alpha=0.3, linewidth=1)

ax2.set_xlabel('X (m)', fontsize=13)
ax2.set_ylabel('Y (m)', fontsize=13)
ax2.set_title('Bo tri mat bang cac tram GNSS (nhin tu tren)\n'
              'Duong net dut: cap East-West cung vi tri',
              fontsize=15, fontweight='bold')
ax2.legend(fontsize=11)
ax2.grid(True, alpha=0.3)
ax2.set_aspect('equal')
plt.tight_layout()
plt.show()